# Linear Algebra of Red Black Graphs

## Introduction
We have provided a formal definition for a Red Black Graph, looked at the adjacency matrix and it's transitive closure. We've discussed *pedigree numbers* and the *avos* product for scalers and looked at a number of interesting properites derived. We'll now extend these observations into a more general discussion of how principles of linear algebra can be applied to Red Black Graphs.

## Vector Classes
Within the context of a Red Black Graph and it's adjacency matrix, we define the following vector classes:

* *row* vector - represented by $\vec u$. These vectors represent ancestry for a given vertex. Values for elements in these vectors are constrained to all positive integers, 0 and 1. Any non-zero integer may appear in an element at most once. 
* *column* vector - represented by $\vec v$. These vectors represent descendency for a given vertex. Values for elements in these vectors are also constrained to all positive integers, 0 and 1. Furthermore if the vertex is red, they are constrained to even numbers and -1, if the vertex is black, they are constrained to odd numbers (plus zero). 1 or -1 may appear only once, otherwise there are no constraints. 
* *simple row vector* - represented by $\vec u_{s}$. Row vectors for which elements are constrained to {-1, 0, 1, 2, 3}. These represent a given vertex and it's immediate ancestry only.
* *simple column vector* - represented by $\vec v_{s}$. Column vectors for which elements are constrained to {-1, 0, 1, 2, 3}. These represent a given vertex and it's immediate descendency only.
* *closed row vector* - represented by $\vec u_{c}$. Row vectors from $A_{rb}^+$. These represent the complete ancestry for a given vertex.
* *closed column vector* - represented by $\vec v_{c}$. Column vectors from $A_{rb}^+$. These represent the complete descendency for a given vertex.

## *avos* Product for Vectors

Let's consider what an *avos* product might represent. Given a row vector and a column vector, we'll define the *avos* product to yield the *pedigree number* representing the relationship between the vertices representing the row and column vectors respectively. 

Consider the Red Black adjacency matrix from our prior example:

$$\begin{bmatrix}
-1 & 2 & 3 & 0 & 0 \\
0 & -1 & 0 & 2 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & -1 & 0 \\
2 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$$

Assuming 0 indexing in the tha matrix, the 4th row vector is a simple row vector for $vertex_{4}$ while the 2nd column vector is simple column for $vertex_{2}$. It is trivally observable that that $vertex_{4}$ is related to $vertex_{2}$ (by *pedigree number* 5). 

$$\begin{bmatrix}
2 & 0 & 0 & 0 & 1 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
3 \\
0 \\
1 \\
0 \\
0 \\
\end{bmatrix}
= 5$$

The vector dot product, summing element-wise products, results in a scaler value of 6. Element wise *avos* product does in fact yield *pedigree number* that represents a relationship. However, it is possible for there to be multiple paths through the graph bewteen two nodes. If that were the case, summing the element-wise *avos* product would not result in the *pedigree number* representing the relationship from $vertex_{a}$ to $vertex_{c}$. For the *avos* vector product, rather than summing the element-wise *avos* products we arbitrarily choos the non-zero minimum element-wise product, thus representing the "closest" relationship between $vertex_{a}$ and $vertex_{c}$.

A simple implementation of the nz_min and the *avos* vector product follows:

In [2]:
# %load ../redblackgraph/simple/vec_avos.py
from redblackgraph.simple import avos
from redblackgraph.simple.util import nz_min

def vec_avos(x, y):
    '''Given two vectors, compute the "avos" dot product.'''
    return nz_min([avos(a, b) for a, b in zip(x, y)])


### Observations

The product of a simple row vector and the transitive closure of a Red Black adjacency matrix is a closed row vector
$$\vec u_{s} A_{rb}^+ = \vec u_{c}$$

The product of the transitive closure of a Red Black adjacency matrix and a simple column vector is a closed column vector
$$A_{rb}^+ \vec v_{s} = \vec v_{c}$$

## Matrix multiplication

With scaler and vector *avos* products defined, extension to matrices is elementary. Given *A* and *B*, both matrices following the constraints defined for Red-Black adjacency matrices, and *C* = *A* *B*, the elements of $C_{ij}$ are given by the vector *avos* product of *row vector<sub>i</sub>* from A and *column vector<sub>j</sub>* from B

Matrix multiplication of Red-Black adjacency matrices may seem a little abstract so let's consider a practical example to motivate the analysis. Given a Red-Black adjacency matrix, *A<sub>rb</sub>*, the result of $A_{rb}^2$ shows all vertices directly related by following up to 2 relationship edges, the result of $A_{rb}^3$ shows all vertices related by following up to 3 relationship edges, etc. For some *n* <= |*V*| there will be a $A_{rb}^n$ == $A_{rb}^+$.

Another way to conceptualize this is that the *avos* matrix product is the just a matrix product with multiplication replaced by the *avos* product and summation replaced by nz_min.

A simple implementation of the avos matrix product follows:

In [3]:
# %load ../redblackgraph/simple/mat_avos.py
from redblackgraph.simple import avos
from redblackgraph.simple.util import nz_min


def mat_avos(A, B):
    '''Given two matrices, compute the "avos" product.'''
    return [[nz_min([avos(a, b) for a, b in zip(A_row, B_col)]) for B_col in zip(*B)] for A_row in A]


## Relational Composition

Consider adding a vertex to the graph that is not currently represented and how we might update $A_{rb}^+$ as a result of the vertex addition. In order to introduce a vetex, $\lambda$, into the graph, that vertex may connect to edges and may also have edges connect to it. This is represented by $\vec u_{\lambda, s}$, a row vector that captures direct edges from $\lambda$ to existing vertices, and $\vec v_{\lambda, s}$, a column vector that captures direct edges from existing vertices to $\lambda$. Note however, that if $A_{rb}^+$ has dimension **N x N**, then $\vec v_{\lambda, s}$ and $\vec u_{\lambda, s}$ have dimension **N + 1**, where the final element for either $\vec u_{\lambda, s}$ and $\vec v_{\lambda, s}$ will either be -1 or 1 depending on whether the added vertex is red or black. Finally, we'll denote $A_{\lambda, rb}^+$ as the transitive closure of the Red Black adjacency matrix of $A_{rb}$ with vetex $\lambda$ added. We'll represent this notationally by:

$$A_{\lambda, rb}^+ = \vec u_{\lambda, s} A_{rb}^+ \vec v_{\lambda, s}$$

and designate this relationship as the **Relational Composition** of the transitive closure of a Red Black adjacency matrix.

As we consider the elementary operations necessary for this composition, recall our observation of the *avos* vector product, namely that the product of a simple row vector and the transitive closure of a Red Black adjacency matrix results in a closed row vector, and that the product of the transitive closure of a Red Black adjacency matrix and a simple column vector results in a closed column vector. If we remove the final element of $\vec v_{\lambda, s}$ and $\vec u_{\lambda, s}$ and apply this observation and then add in the final element to the resulting vectors, the result is $\vec u_{\lambda, c}$ and $\vec v_{\lambda, c}$. The complete row and column vectors can be appended as a new row and column respectively to $A_{rb}^+$. 

This new matrix does not quite represent the **relational composition**. In order to ensure that it is transitively closed we need to examine each existing row and for each element where $\vec v_{\lambda, c}$ is non-zero, we need to update the element in the given row to the *avos* product of the column element in $\vec v_{\lambda, c}$ and the row element of the corresponding row in $\vec u_{\lambda, c}$.

Expressing this algorithmically, given $A_{rb}^+$, $\vec u_{\lambda, s}$ and $\vec v_{\lambda, s}$:

1. Remove the "last" dimension from both $\vec u_{\lambda, s}$ and $\vec v_{\lambda, s}$ resulting in $\vec u'_{\lambda, s}$ and $\vec v'_{\lambda, s}$
2. generate $\vec u'_{\lambda, c}$ by $\vec u_{\lambda, s}$ $A_{rb}^+$ and $\vec v'_{\lambda, c}$ by $A_{rb}^+$ $\vec v_{\lambda, s}$
3. Compose $A'_{\lambda, rb}$ by:
    1. appending $\vec u'_{\lambda, c}$ to $A_{rb}^+$ as a new row 
    2. Add the "last" dimension removed in step 1 as the "last" dimension of $\vec u'_{\lambda, c}$ resulting in $\vec u_{\lambda, c}$
    3. appending $\vec v_{\lambda, c}$ to $A_{rb}^+$ as a new column resulting in $A'_{lamba,rb}$
    4. For each row, $i$, where $\vec u_{c}[j]$ != 0, set $A'_{lamba,rb}[i,j] = \vec u_{c}[j]$ *avos* $\vec v_{c}[i]$
    
A simple implementation of the relational composition follows:

In [ ]:
# %load ../redblackgraph/simple/rel_composition.py
from redblackgraph.simple import avos, mat_avos
from redblackgraph.simple.util import nz_min
import copy


def relational_composition(u, A, v):
    '''
    Given simple row vector u, transitively closed matrix A, and simple column vector v where
    u and v represent a vertex, lambda, not currently represented in A, compose A_{\lambda} wich is
    the transitive closure for the graph with lambda included
    :param u: simple row vector for new vertex, lambda
    :param A: transitive closure for Red Black graph
    :param v: simple column vector for new vertex, lambda
    :return: transitive closure for Red BLack graph with lambda
    '''
    N = len(A)
    uc_lambda = mat_avos([u[0][:-1]], A)
    vc_lambda = mat_avos(A, v[:-1])
    A_lambda = copy.deepcopy(A)
    A_lambda.append(uc_lambda[0])
    for i in range(N):
        A_lambda[i].append(vc_lambda[i][0])
        for j in range(N):
            if not uc_lambda[0][j] == 0:
                A_lambda[i][j] = nz_min(avos(vc_lambda[i][0], uc_lambda[0][j]), A_lambda[i][j])
    A_lambda[N].append(u[0][N])
    return A_lambda


## Loop Prevention

An issue that can be encountered in systems that represent familial relationships is the inadvertent injection of graph cycles, resulting in the ["I am my own Grandpa"](https://en.wikipedia.org/wiki/I%27m_My_Own_Grandpa) case. While this is impossible when relationships model sexual reproduction, the introduction of step-relationships would make this a possibility. Often times there is ambiguity in the available historical records. If a researcher isn't careful, cylces may result.

A modification to the relational_composition algorithm can prevent the introduction of cycles into a Red Black Graph. Vertices are added to an existing graph via relational composition. The intermedite vectors, $\vec u'_{\lambda, c}$ and $\vec v'_{\lambda, s}$ represent the complete ancestry and complete descedency for the new vertex $\lambda$ respectively. The cycle consstraint would be invalidated should there be any vertex that simultaneously appears in the ancestry and descendency for a given vertex.

This is easily validated in the **relational composition** algorithm and can be expressed more formally. Given $\vec u'$ and $\vec v'$ of dimension $n$ defined as $u'_{\lambda, c}$ and $v'_{\lambda, c}$ above. **Relational composition** is undefined if there exists a dimension $i$ where $i$ != $n$ and $u'_{i}$ != 0 and $v'_{i}$ != 0. The **relational composition** is well-formed otherwise.

## Determinants

Let's explore the determinants of the class of matrices that represent Red Black Graphs. Staring with the simple case of a $2 x 2$ matrix.

$$\begin{vmatrix}A\end{vmatrix} = \begin{vmatrix}a & b \\ c & d \\ \end{vmatrix} = ad - cb$$

As per formal definition, $a$ and $d$ $\in \left\{ {-1, 1}\right\}$; $b$ defines the relationship from the vertex represented by the first row to the vertex represented by the second row; $c$ defines the relationship from the vertex represented by the second row to the vertex represented by the first row. 

As per constraints (no cycles) if $b$ is non-zero then $c$ must be zero and conversely if $c$ is non-zero, $b$ must be zero.

Now the question is presented as to what arithmetic operators should be used in the calculation of $\begin{vmatrix}A\end{vmatrix} = ad - cb$. If, as in matrix multiplication, we use the *avos* operator in place of multiplication the the term $ad$ will also be constrained to $\left\{ {-1, 1}\right\}$. If instead we use multiplication, $ad$ will still be constrained to $\left\{ {-1, 1}\right\}$. The term $cb$ will always be 0. Whether we use addition/subtraction or non-zero-min to combine the terms $ad$ and $cb$ is immaterial as the $cb$ is 0 and the result will be determined by $ad$. 

It's interesting to also look at the case of a $3 x 3$ matrix.

$$\begin{vmatrix}A\end{vmatrix} = \begin{vmatrix}a & b & c \\ d & e & f \\ g & h & i \end{vmatrix} = aei + bfg + cdh - ceg - bdi - afh$$

As in the $2 x 2$ case, the product of the diagonals is constrained to $\left\{ {-1, 1}\right\}$ and all other terms will be zero as they either represent the cycle of path length 2 or path length 1. Let's label the vertex representing by the first row as $\alpha$, the second row as $\beta$ and the third row as $\gamma$. Let's look at the $bfg$ term. $b$ represents the relationship from $\alpha$ to $\beta$, $f$ represents the relationship from $\beta$ to $\gamma$ and $g$ represents the relationship from $\gamma$ to $\alpha$. This term defines a cycle of path length 2 and at least one of the terms must be zero by constraint. Again, successive application of *avos* or multiplication will result in zero. 

Let's look at the $ceg$ term. $c$ represents the relationship from $\alpha$ to $\gamma$, $e$ represents the relationship from $\beta$ to itself and $g$ represents the relationship from $\gamma$ to $\alpha$. Again, by constraint, either $c$ or $g$ must be zero. Therfore $ceg$ will be zero.

(I'm weak at formal proofs, but by taking the above reasoning, and applying it to matrix minors and cofactors it can be shown that matrices that represent Red Black graphs are Unimodular.)

The determinant of a Red Black graph matrix will be $\left\{ {-1, 1}\right\}$.


## Areas of Further investigation

* Eigenvalues
* *avos* properties (commutitivity, associativity, distribution, identity, inverse, etc.)
* Spectral Graph Theory
* Explore ideas to reduce dimensionality (embedding similar to word2vec, etc.)
* Axioms of linear algebra:
    * u + (v + w) = (u + v) + w
    * v + w = w + v
    * There is a vector *0* such that *0* + v = v for all v
    * For every vector v there is a vector -v so that v + (-v) = *0*
    * a(bv) = (ab)v
    * 1v = v
    * a(v + w) = av + aw
    * (a + b)v = av + bv